In [1]:
from image_explainer import *
from utils2 import *
import matplotlib.pyplot as plt
import matplotlib.transforms as mtrans
import numpy as np
import pandas as pd
import torch
import torchvision
from sklearn.manifold import TSNE
from modules import *
import random
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
from sklearn.cluster import SpectralClustering
import community

In [25]:
embeds = load_inception_embeds()
X_train = torch.tensor(embeds["X_train"])
Y_train = torch.tensor(embeds["Y_train"])

X_test =  torch.cat((torch.tensor(embeds["X_test"]), torch.tensor(embeds["X_train"])), dim=0)
Y_test = torch.cat((torch.tensor(embeds["Y_test"]), torch.tensor(embeds["Y_train"])), dim=0)

# X_test = torch.tensor(embeds["X_test"])
# Y_test = torch.tensor(embeds["Y_test"])

train_set = data.TensorDataset(X_train, Y_train)
test_set = data.TensorDataset(X_test, Y_test)
clf = fit_model(X_train, Y_train)

Using cached Inceptionv3 embeddings: /user/cs.aau.dk/em63by/Influence_Function_Evaluation/Projects/data/dataset_coda.npz


In [26]:
module = LiSSAInfluenceModule(
    model=clf,
    objective=BinClassObjective(),
    train_loader=data.DataLoader(train_set, batch_size=32),
    test_loader=data.DataLoader(test_set, batch_size=32),
    device=DEVICE,
    damp=0.001,
    repeat= 1,
    depth=1800,
    scale= 10,
)
train_idxs = list(range(X_train.shape[0]))

In [ ]:
k=120
fig, axs = plt.subplots(1, 10, figsize=(11, 11))
for i, j in enumerate(weirdos[k:k+10]):
    axs[i].imshow(captioned_image(clf, embeds, 'train', j-600))
    axs[i].axis('off')
plt.show()

#### Influence Analysis

In [ ]:
[i for i in range(len(Y_test)) if module.influences(train_idxs=train_idxs, test_idxs=[i]).sum()==0]

In [24]:
len([Y_test[i] for i in weirdos])

IndexError: index 627 is out of bounds for dimension 0 with size 600

In [ ]:
1284, 2374, 2257, 311, 1720, 1180, 2205     ['ex-1657', 'ex-84', 'ex-1359', 'ex-986', 'ex-1687']

In [28]:
weirdos=[8, 29, 48, 55, 57, 60, 65, 85, 98, 123, 134, 137, 175, 183, 189, 215, 232, 264, 269, 309, 311, 316, 344, 358, 366, 372, 390, 403, 420, 424, 430, 434, 469, 482, 496, 522, 550, 582, 627, 648, 655, 674, 709, 712, 720, 721, 728, 733, 734, 738, 766, 774, 777, 788, 799, 802, 804, 810, 830, 886, 896, 930, 932, 954, 955, 1006, 1007, 1014, 1015, 1030, 1036, 1046, 1051, 1053, 1062, 1099, 1116, 1118, 1128, 1138, 1140, 1143, 1146, 1147, 1172, 1180, 1192, 1227, 1253, 1255, 1271, 1284, 1285, 1357, 1360, 1369, 1372, 1400, 1401, 1408, 1413, 1419, 1422, 1432, 1451, 1483, 1502, 1514, 1524, 1531, 1570, 1586, 1598, 1617, 1631, 1636, 1675, 1677, 1686, 1699, 1713, 1720, 1722, 1752, 1800, 1816, 1819, 1823, 1848, 1879, 1884, 1890, 1913, 1940, 1951, 1952, 1959, 1972, 1997, 1998, 2000, 2001, 2014, 2036, 2039, 2057, 2078, 2090, 2110, 2131, 2140, 2141, 2147, 2155, 2162, 2184, 2192, 2194, 2200, 2201, 2205, 2210, 2229, 2232, 2239, 2250, 2257, 2268, 2272, 2277, 2287, 2296, 2297, 2303, 2343, 2361, 2374]

In [34]:
clf(X_test[6].unsqueeze(0)).item()

0.9999957084655762

In [31]:
[i for i in range(len(Y_test)) if clf(X_test[i].unsqueeze(0)).item()==Y_test]

[]

# 1. Graph generation

In [5]:
# Build a dataframe with X-trains with threir corresponding influence scores and similarity
def df_construct(test_idx, train_idxs):
    influences = module.influences(train_idxs=train_idxs, test_idxs=[test_idx])
    
    similarity=[cosine_similarity(X_test[test_idx].numpy().reshape(1,-1), X_train[i].numpy().reshape(1, -1)).item()
               for i in range(len(X_train))]

    data = {'Influence': influences.reshape(-1).tolist(),'Similarity': similarity, 'Y_train':Y_train.tolist(), 'X_train':X_train.numpy().tolist()}
    df = pd.DataFrame(data)
    return df
def get_explanation(i):
    
        df = df_construct(i, train_idxs)
        df_pos_sl = input_data(df, i, Y_test)
        selected_indices_pos_sl = greedy_subset_selection(df_pos_sl, N=5)
        a={df_pos_sl.Influence.index[k]:df_pos_sl.Influence.tolist()[k] for k in selected_indices_pos_sl}
        return a
    
def generate_influential_samples(t, aide=False):
    if aide or module.influences(train_idxs=train_idxs, test_idxs=[t]).sum().item()==0:
        return get_explanation(t)
    else:
        scaler = MinMaxScaler()
        inf_list=scaler.fit_transform(module.influences(train_idxs=train_idxs, test_idxs=[t]).reshape(-1, 1))
        inf_list=torch.tensor(inf_list.squeeze()).sort(descending=True)
        return {inf_list[1][i].item(): inf_list[0][i].item() for i in range(5)}
    
def draw_explanation(test_indx, aide=False):
    # ===========
    # Plot image
    # =========
    new_line = '\n'
    fig, axs = plt.subplots(2, 5, figsize=(18, 8))
    if test_indx>600:
        axs[0, 0].imshow(captioned_image(clf, embeds, 'train', test_indx-600))
    else:
        axs[0, 0].imshow(captioned_image(clf, embeds, 'test', test_indx))
    axs[0, 0].axis('off')


    exlist=generate_influential_samples(test_indx, aide=aide).keys()
    for i, j in enumerate(exlist):
        if i<5:
            axs[1, i].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[1, i].axis('off')
            axs[0, i].axis('off')
        else:
            axs[2, i-5].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[2, i-5].axis('off')
            axs[0, i-5].axis('off')

    plt.show()

def plotim(i):
    if i<600:
        image = captioned_image(clf, embeds, 'test', i)
        plt.imshow(image)
        plt.axis('off')
        plt.show()
    else:
        image = captioned_image(clf, embeds, 'train', i-600)
        plt.imshow(image)
        plt.axis('off')
        plt.show()

In [ ]:
# List of T's
points_T = [i for i in range(len(Y_test))]

# Create influential_samples dictionary automatically
influential_samples = {t: generate_influential_samples(t) for t in tqdm(points_T)}


# Create a bipartite graph
G = nx.Graph()

# Dummy image tensors for illustration

# Create a bipartite graph
G = nx.Graph()

# Add nodes from sets T and E with vector representations and image tensors
for i in points_T:
    G.add_node(i, bipartite=0, emd=X_test[i].numpy())

for point, samples in influential_samples.items():
    for sample, score in samples.items():
        G.add_node(f'ex-{sample}', bipartite=1, emd=X_train[sample].numpy())
        G.add_edge(point, f'ex-{sample}', weight=score)

# Save the graph as data
nx.write_gpickle(G, 'bipartite_graph.pkl')

In [6]:
G = nx.read_gpickle('/data/ikhtiyor/bipartite_graph_5.pkl')

# 2. Centrality check

In [ ]:
# Calculate degree centrality for each node in the graph
degree_centrality = nx.degree_centrality(G)

k = 10  # Adjust as needed
top_k_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:k]

# Print top k nodes with their degrees
print(f"Top {k} nodes with highest degree centrality:")
for node in top_k_nodes:
    print(f"Node {node}: Degree {G.degree(node)}")
    

# fig, axs = plt.subplots(1, k, figsize=(11, 11))
# for i, j in enumerate(top_k_nodes):
#     axs[i].imshow(captioned_image(clf, embeds, 'train', int(j.split('-')[1])))
#     axs[i].axis('off')
# plt.show()

In [ ]:
# degree_centrality
degrees=[G.degree(i) for i in G.nodes if G.nodes[i]['bipartite']==1]

In [ ]:
max(degrees)

In [ ]:
plt.hist(degrees, bins=130)
plt.xticks(fontsize=14)
plt.ticklabel_format(style='sci', axis='x')
plt.yticks(fontsize=13)
plt.xlabel("Centrality Degree", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
# plt.savefig('inf_cnn_sortsum.eps', format='eps')
plt.show()

#### TSNE of high degree points

In [ ]:
from sklearn.manifold import TSNE

# Initialize the t-SNE algorithm
tsne = TSNE(n_components=2, random_state=42)

# Combine the training and test tensors
X = torch.cat((X_train, X_test), dim=0)
y = torch.cat((Y_train, Y_test), dim=0)

# Convert the tensors to numpy arrays
X = X_test.numpy()
y = Y_test.numpy()

# Initialize the t-SNE algorithm
tsne = TSNE(n_components=2, random_state=0)

# Fit and transform the data to 2D
X_2d = tsne.fit_transform(X)

# Plot the data in 2D with different colors and alpha values for each class
plt.scatter(X_2d[y==0, 0], X_2d[y==0, 1], marker='o', c='#7FFFD4',  label='Dog', alpha=0.7)
plt.scatter(X_2d[y==1, 0], X_2d[y==1, 1], marker='o', c='#FFE5B4',  label='Fish', alpha=0.5)
for i in top_k_nodes:
    plt.scatter(X_2d[int(i.split('-')[1])+600][0], X_2d[int(i.split('-')[1])+600][1], marker='.', color='red')
# for i in weirdos:
#     plt.scatter(X_2d[i][0], X_2d[i][1], marker='.', color='red')
plt.legend(fontsize=7)
# plt.savefig(f'tsne_aide{test_idx}.pdf', format='pdf')
plt.show()

# 3. Louvain algorithm

In [7]:
nodes_bipartite_0 = [node for node in G.nodes if G.nodes[node]['bipartite'] == 0]

# Calculate cosine similarity between attribute vectors of nodes with bipartite=0
attribute_matrix = [G.nodes[node]['embedding'] for node in nodes_bipartite_0]
cosine_sim_matrix = cosine_similarity(attribute_matrix, attribute_matrix)

# Create a graph with nodes from bipartite=0 and weighted edges based on cosine similarity
weighted_projection = nx.Graph()
weighted_projection.add_nodes_from(nodes_bipartite_0)

# Add weighted edges based on cosine similarity scores and the neighbours
for i, node1 in enumerate(nodes_bipartite_0):
    for j, node2 in enumerate(nodes_bipartite_0):
        if i < j:
            if len(set(G.neighbors(node1)).intersection(set(G.neighbors(node2))))!=0:
                weight=0
                for k in set(G.neighbors(node1)).intersection(set(G.neighbors(node2))):
                    
#                     if G[i][k]['weight'] + G[j][k]['weight'] ==0:
                    weight = weight+G[i][k]['weight'] + G[j][k]['weight']
#                     else:
                        
#                         weight = cosine_sim_matrix[i][j]+2*G[i][k]['weight']*G[j][k]['weight']/(G[i][k]['weight'] + G[j][k]['weight'])
#             else:
                weight=weight+cosine_sim_matrix[i][j]
                weighted_projection.add_edge(node1, node2, weight=weight)

            

# Run the Louvain algorithm on the weighted projection
partition = community.best_partition(weighted_projection, resolution=2)

# Draw the network with node colors based on community
# pos = nx.spring_layout(weighted_projection, seed=42)
# plt.figure(figsize=(8, 6))
# nx.draw_networkx_nodes(weighted_projection, pos, node_color=list(partition.values()), cmap=plt.cm.Set1, node_size=20)
# nx.draw_networkx_edges(weighted_projection, pos, alpha=0.5)
# plt.title("Community Detection Weighted Louvain")
# plt.axis("off")
# plt.show()

# Print the nodes and their assigned communities in the weighted projection
# coms=[]
# for node, community_id in partition.items():
#     coms.append(community_id)
#     print(f"Node {node}: Community {community_id}: {Y_test[node]}")


# Evaluation and comparison

In [ ]:
import networkx as nx
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score, silhouette_score
from networkx.algorithms.community import quality

# Convert the partition to a list of sets representing communities
communities = [set([node for node, com_id in partition.items() if com_id == community_id]) for community_id in set(partition.values())]

# Modularity
modularity = quality.modularity(weighted_projection, communities)
print(f"Modularity: {modularity}")

In [8]:
df = pd.DataFrame({'Node': partition.keys(), 'Community': partition.values(), 'Label':Y_test}, index=None)
# df.to_csv('communities.scv')
# df=pd.read_csv('communities.scv')

In [9]:
set(df.Community)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [ ]:
degr=[]
for i in range(len(coms)):
    for j in range(len(coms)):
        if i<j and coms[i]==coms[j]:
            print(f"Community {coms[i]}: {i},{j} => {len(set(weighted_projection.neighbors(i)).intersection(set(weighted_projection.neighbors(j))))}")
            degr.append(len(set(weighted_projection.neighbors(i)).intersection(set(weighted_projection.neighbors(j)))))

In [ ]:
df[df.Community==20][:50]

In [ ]:
df.Community.value_counts()

In [10]:
def repr_points(cn):
    # Create a subgraph for the community
    com=[i for i in df.Node if df.Community[i]==cn]
    subgraph = weighted_projection.subgraph(com)

    # Apply Louvain community detection algorithm
    partition = community.best_partition(subgraph)

    # Get representative points from each cluster
    representatives = set()
    for cluster_id in set(partition.values()):
        # Find a representative node for each cluster (e.g., the node with the highest degree)
        nodes_in_cluster = [node for node, c_id in partition.items() if c_id == cluster_id]
        representative_node = max(nodes_in_cluster, key=subgraph.degree)
        representatives.add(representative_node)

    # Print or use the representative nodes
    print("Representative Nodes:", representatives)
    for i in representatives:
        plotim(i)
    return representatives

def globex(cn):
    lst=[]
    for i in df.Node:
        if df.Community[i]==cn:
            lst+=[i for i in G.neighbors(i)]
    return pd.Series(lst).value_counts()[:5].keys()

def coverage(cn):
    com=[i for i in df.Node if df.Community[i]==cn]
    gx=globex(cn)
    return len([i for i in com if len(set(gx) & set(G.neighbors(i)))>0])


In [ ]:
for i in globex(0):
    image = captioned_image(clf, embeds, 'train', int(i.split('-')[1]))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
repr_points(0)

In [ ]:
len(set(df[df['Community']==6]['Node'])&set(weirdos))

## Accuracy of the prototype distance-based models

In [19]:
exs=[]
for i in set(df.Community):
    exs+=globex(i).tolist()

In [20]:
len(exs)

80

In [22]:
accr=0
for i in tqdm(range(len(Y_test))):
    if Y_test[i]==Y_train[int(exs[np.array([cosine_similarity(X_test[i].reshape(1, -1), X_train[int(j.split('-')[1])].reshape(1, -1)) for j in exs]).argmax()].split('-')[1])]:
        accr+=1
accr/len(Y_test)

100%|█████████████████████████████████████████████████████████████| 600/600 [00:19<00:00, 30.93it/s]


0.94

In [17]:
exsota=[]
for i in set(df.Community):
    exsota+=globexsota(i).tolist()

In [18]:
len(exsota)

80

In [23]:
accrsota=0
for i in tqdm(range(len(Y_test))):
    if Y_test[i]==Y_train[int(exsota[np.array([cosine_similarity(X_test[i].reshape(1, -1), X_train[int(j.split('-')[1])].reshape(1, -1)) for j in exsota]).argmax()].split('-')[1])]:
        accrsota+=1
accrsota/len(Y_test)

100%|█████████████████████████████████████████████████████████████| 600/600 [00:19<00:00, 30.93it/s]


0.9166666666666666

In [ ]:
covs=[]
membrs=[]
for i in range(22):
    covs.append(coverage(i)/df.Community.value_counts()[i])
    membrs.append(df.Community.value_counts()[i])
    print(f"Coverage: {coverage(i)/df.Community.value_counts()[i]},  Members: {coverage(i)}/{df.Community.value_counts()[i]}")

In [ ]:
plt.scatter(membrs, covs)
plt.ylabel('Coverage')
plt.xlabel('Members')
plt.legend()
plt.show()

In [ ]:
sum([coverage(i) for i in range(23)])/2400

In [ ]:
avgs=[]
comss=[]
for i in range(22):
    a=X_test[[df[df['Community']==i]['Node'].tolist()]]
    cos_sim_matrix = cosine_similarity(a, a)
    # Extract the upper triangular part of the cosine similarity matrix (excluding the diagonal)
    cos_sim_values = cos_sim_matrix[np.triu_indices(cos_sim_matrix.shape[0], k=1)]

    # Calculate the average cosine similarity
    average_cosine_similarity = np.mean(cos_sim_values)
    avgs.append(average_cosine_similarity)
    comss.append(len(a))
    print(f"Average Cosine Similarity of community {i} - {len(a)}:", average_cosine_similarity)

In [ ]:
sum(avgs)/23

In [ ]:
plt.scatter(comss, avgs)
plt.ylabel('Average Cosisne')
plt.xlabel('Members')
plt.legend()
plt.show()

In [ ]:
plt.scatter(avgs, covs)
plt.ylabel('Coverage')
plt.xlabel('Avg cosine')
plt.legend()
plt.show()

In [12]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import numpy as np

def calculate_silhouette_score(data, cluster_assignments):
    """
    Calculate the silhouette score for a clustering.

    Parameters:
    - data: The input data.
    - cluster_assignments: A list or array containing the cluster assignment for each data point.

    Returns:
    - silhouette_avg: The average silhouette score.
    - silhouette_values: A list of silhouette scores for each data point.
    """
    silhouette_avg = silhouette_score(data, cluster_assignments)
    silhouette_values = silhouette_samples(data, cluster_assignments)
    return silhouette_avg, silhouette_values

# Example usage:
# Assuming you have your data (X) and cluster assignments (labels) ready
# Replace X and labels with your actual data and cluster assignments
# For example, if you're using KMeans clustering from scikit-learn:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=3)
# labels = kmeans.fit_predict(X)

silhouette_avg, silhouette_values = calculate_silhouette_score(X_test, df.Community.tolist())

print("Silhouette Score:", silhouette_avg)

Silhouette Score: -0.0018559345


In [15]:
# Example usage:
# Assuming you have your data (X) and cluster assignments (labels) ready
# Replace X and labels with your actual data and cluster assignments
# For example, if you're using KMeans clustering from scikit-learn:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=16)
labels = kmeans.fit_predict(X_test)

silhouette_avg, silhouette_values = calculate_silhouette_score(X_test, labels)

print("Silhouette Score:", silhouette_avg)

Silhouette Score: 0.018828157


In [16]:
def globexsota(cn):
    lst=[]
    for i in df.Node:
        if labels[i]==cn:
            lst+=[i for i in G.neighbors(i)]
    return pd.Series(lst).value_counts()[:5].keys()
def coveragesota(cn):
    com=[i for i in df.Node if labels[i]==cn]
    gx=globexsota(cn)
    return len([i for i in com if len(set(gx) & set(G.neighbors(i)))>0])

In [ ]:
covs=[]
membrs=[]
for i in range(23):
    covs.append(coveragesota(i)/pd.Series(labels).value_counts()[i])
    membrs.append(pd.Series(labels).value_counts()[i])
    print(f"Coverage: {coveragesota(i)/pd.Series(labels).value_counts()[i]},  Members: {coveragesota(i)}/{pd.Series(labels).value_counts()[i]}")

In [ ]:
sum([coveragesota(i) for i in range(i)])/2400

In [ ]:
plt.scatter(membrs, covs)
plt.ylabel('Coverage')
plt.xlabel('Members')
plt.legend()
plt.show()

In [ ]:
labels.shape[0]

In [ ]:
avgs=[]
comss=[]
for i in range(23):
    a=X_test[[j for j in range(labels.shape[0]) if labels[j]==i]]
    cos_sim_matrix = cosine_similarity(a, a)
    # Extract the upper triangular part of the cosine similarity matrix (excluding the diagonal)
    cos_sim_values = cos_sim_matrix[np.triu_indices(cos_sim_matrix.shape[0], k=1)]

    # Calculate the average cosine similarity
    average_cosine_similarity = np.mean(cos_sim_values)
    avgs.append(average_cosine_similarity)
    comss.append(len(a))
    print(f"Average Cosine Similarity of community {i} - {len(a)}:", average_cosine_similarity)

In [ ]:
sum(avgs)/23

In [ ]:
plt.scatter(comss, avgs)
plt.ylabel('Similarity')
plt.xlabel('Members')
plt.legend()
plt.show()

In [ ]:
image = captioned_image(clf, embeds, 'train', 687)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
repr_points(802)

In [ ]:
repr_points(11)

In [ ]:
repr_points(0)

In [ ]:
df[df['Community']==21].head(50)

In [ ]:
repr_points(21)

In [ ]:
repr_points(8)

In [ ]:
repr_points(13)

In [ ]:
repr_points(10)

#### Representative points through k-mediods

In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

# Step 1: Compute pairwise distances between embeddings
distances = pairwise_distances(X_test[com5], metric='euclidean')

# Step 2: Use KMedoids algorithm to cluster the embeddings
k = 10  # Replace with your desired number of clusters
kmedoids = KMedoids(n_clusters=k, random_state=0, metric='precomputed')
clusters = kmedoids.fit_predict(distances)

# Step 3: Identify medoids as representative points
medoids_indices = kmedoids.medoid_indices_

# Example: Assuming 'image_embeddings' is a numpy array
representative_points = X_test[com5][medoids_indices]

# You can visualize the clusters and medoids if needed
plt.scatter(X_test[com5][:, 0], X_test[com5][:, 1], c=clusters, cmap='viridis')
plt.scatter(representative_points[:, 0], representative_points[:, 1], marker='X', c='red', s=200, label='Medoids')
plt.legend()
plt.show()

In [ ]:
medoid_distances = pairwise_distances(X_test[com5][medoids_indices], metric='euclidean')
diversity_scores = np.min(medoid_distances + np.eye(k) * np.max(medoid_distances), axis=0)
diverse_indices = np.argsort(diversity_scores)[-k:][::-1]
diverse_medoids = medoids_indices[diverse_indices]

In [ ]:
cluster_diversity = [np.mean(distances[cluster][:, medoids_indices[cluster]]) for cluster in range(k)]
representative_points = [cluster[medoids_indices[cluster]] for cluster in clusters]


In [ ]:
diverse_medoids

In [ ]:
medoids_indices

In [ ]:
com5[38]

In [ ]:
for
set(weighted_projection.neighbors(5)).intersection(set(weighted_projection.neighbors(13)))

In [ ]:
draw_explanation(263, aide=True)

In [ ]:
draw_explanation(1480, aide=True)

In [ ]:
draw_explanation(1460, aide=True)

In [ ]:
draw_explanation(565, aide=True)

In [ ]:
draw_explanation(606, aide=True)

In [ ]:
# draw_explanation(1120, aide=True)

In [ ]:
image = captioned_image(clf, embeds, 'train', 558)
plt.imshow(image)
plt.axis('off')
plt.show()

# 4. Bispectral clustering

In [ ]:
points_T = [i for i in range(len(Y_test))]
# Filter nodes and get the biadjacency matrix for points_T
# biadjacency_mat = bipartite.biadjacency_matrix(G)

# Convert to a regular adjacency matrix
# adjacency_mat = biadjacency_mat @ biadjacency_mat.T
adjacency_mat = nx.adjacency_matrix(weighted_projection).toarray()
# Calculate cosine similarity between node attributes for points_T
# attribute_matrix = np.array([G.nodes[node]["embedding"] for node in points_T])
attribute_matrix = np.asarray([G.nodes[node]["embedding"] for node in points_T])
cosine_sim_matrix = cosine_similarity(attribute_matrix, attribute_matrix)

# Combine adjacency matrix and cosine similarity matrix
combined_matrix = adjacency_mat + cosine_sim_matrix

# Perform spectral clustering using scikit-learn
num_communities = 16 
print('until here')
spectral = SpectralClustering(n_clusters=num_communities, affinity='precomputed', random_state=42, n_jobs=1)

# from sklearn.decomposition import PCA

# Reduce dimensionality using PCA
# pca = PCA(n_components=100)  # Choose an appropriate number of components
# combined_matrix_pca = pca.fit_transform(combined_matrix)


communities = spectral.fit_predict(adjacency_mat)
df = pd.DataFrame({'Node': points_T, 'Community': communities, 'Label':Y_test}, index=None)

# # Save the DataFrame to a CSV file
df.to_csv('communities.csv', index=False)

In [ ]:
image = captioned_image(clf, embeds, 'test', 458)
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
df.value_counts("Community")

In [ ]:
df[df["Community"]==18]

In [ ]:
draw_explanation(802, aide=True)

In [ ]:
draw_explanation(2115, aide=True)

In [ ]:
cosine_similarity(X_test[5].numpy().reshape(1,-1), X_train[0].numpy().reshape(1, -1))

In [ ]:
draw(7, aide=True)

In [ ]:
Y_test[5]

In [ ]:
module.influences(train_idxs=train_idxs, test_idxs=[802]).sort(descending=True)

In [ ]:
module.influences(train_idxs=train_idxs, test_idxs=[8]).sum().item()==0

In [ ]:
draw(9, aide=True)

In [ ]:
generate_influential_samples(599, aide=True)